# VII. Timing Code

Julia has a built-in macro `@time` that can be used to time code.

In [1]:
using LinearAlgebra

In [2]:
# Exponentiate a matrix

function ExpMat(A,n)
    nr = size(A,1)
    I = Diagonal(ones(nr,nr))
    s = I + A
    t = A
    for j = 2:n
        s += t*A / n
    end
    return s
end

ExpMat (generic function with 1 method)

In [3]:
U = randn(1000,1000);

The first time Julia encounters a user-written function it will compile and then cache this compiled function. Therefore the first very first timing will include the compilation time. To get an accurate timing of the function's execution time you will want to time the function a few times. 

In [4]:
@time ExpMat(U, 20);

  2.343776 seconds (3.93 M allocations: 636.256 MiB, 11.71% gc time)


In [5]:
@time ExpMat(U, 20);

  1.142022 seconds (126 allocations: 450.147 MiB, 1.48% gc time)


In [6]:
@time ExpMat(U, 20);

  1.133800 seconds (126 allocations: 450.147 MiB, 1.32% gc time)


An alternative is to use the __BenchmarkTools__ package which will provide a benchmark based on a sample of runs and provides sample statistics.

In [7]:
using BenchmarkTools

In [8]:
@benchmark ExpMat(U, 20) seconds = 6

BenchmarkTools.Trial: 
  memory estimate:  450.15 MiB
  allocs estimate:  122
  --------------
  minimum time:     1.123 s (0.98% GC)
  median time:      1.142 s (1.32% GC)
  mean time:        1.161 s (3.50% GC)
  maximum time:     1.283 s (13.54% GC)
  --------------
  samples:          6
  evals/sample:     1

We saw before there were three ways to do apply a function to a collection of values: array comprehension, map function, using the dot syntax, and a for loop. We can benchmark these approaches to see how they compare.

In [9]:
array = randn(10^6);

In [10]:
@benchmark [exp(cos(u)) for u in array] samples=20

BenchmarkTools.Trial: 
  memory estimate:  7.63 MiB
  allocs estimate:  3
  --------------
  minimum time:     23.053 ms (0.00% GC)
  median time:      23.204 ms (0.00% GC)
  mean time:        23.617 ms (0.84% GC)
  maximum time:     25.268 ms (8.26% GC)
  --------------
  samples:          20
  evals/sample:     1

In [11]:
@benchmark map(x -> exp(cos(x)), array) samples=20

BenchmarkTools.Trial: 
  memory estimate:  7.63 MiB
  allocs estimate:  3
  --------------
  minimum time:     23.028 ms (0.00% GC)
  median time:      23.353 ms (0.00% GC)
  mean time:        23.655 ms (0.59% GC)
  maximum time:     25.830 ms (3.64% GC)
  --------------
  samples:          20
  evals/sample:     1

In [12]:
@benchmark exp.(cos.(array)) samples = 20

BenchmarkTools.Trial: 
  memory estimate:  7.63 MiB
  allocs estimate:  7
  --------------
  minimum time:     22.486 ms (0.00% GC)
  median time:      22.768 ms (0.00% GC)
  mean time:        23.058 ms (0.64% GC)
  maximum time:     24.700 ms (3.81% GC)
  --------------
  samples:          20
  evals/sample:     1

In [13]:
@benchmark for i in 1:length(array); exp(cos(array[i])); end samples = 20

BenchmarkTools.Trial: 
  memory estimate:  106.80 MiB
  allocs estimate:  5998980
  --------------
  minimum time:     125.953 ms (4.67% GC)
  median time:      128.836 ms (4.71% GC)
  mean time:        129.419 ms (5.55% GC)
  maximum time:     140.446 ms (4.33% GC)
  --------------
  samples:          20
  evals/sample:     1